In [1]:
!wget http://dl.deepnn.ir/Diverse_hand_gesture_dataset.zip

--2024-01-19 16:50:14--  http://dl.deepnn.ir/Diverse_hand_gesture_dataset.zip
Resolving dl.deepnn.ir (dl.deepnn.ir)... 188.40.164.149
Connecting to dl.deepnn.ir (dl.deepnn.ir)|188.40.164.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1088794071 (1.0G) [application/zip]
Saving to: ‘Diverse_hand_gesture_dataset.zip’

Diverse_hand_gestur 100%[===================>]   1.01G  18.6MB/s    in 56s     

2024-01-19 16:51:12 (18.5 MB/s) - ‘Diverse_hand_gesture_dataset.zip’ saved [1088794071/1088794071]



In [2]:
!unzip -q Diverse_hand_gesture_dataset.zip

In [3]:
import os
import shutil

# Set the path to your directory containing PNG and TXT files
input_directory = '/content/train'

# Create a directory to store the organized files
output_directory = 'organized_train'
os.makedirs(output_directory, exist_ok=True)

validation_dir = "validation"

# Define a dictionary to map numeric class numbers to strings
class_mapping = {
    0: 'zero',
    1: 'dislike',
    2: 'five',
    3: 'exactly',
    4: 'two',
    5: 'three',
    6: 'left',
    7: 'like'
}

num = 0
# 10% validation

# Loop through each TXT file in the input directory
for txt_file in os.listdir(input_directory):
    if txt_file.endswith('.txt'):
        txt_path = os.path.join(input_directory, txt_file)

        # Read the class information from the first column of the TXT file
        with open(txt_path, 'r') as file:
            class_number = int(file.readline().strip().split()[0])

        # Map the numeric class number to the corresponding string
        class_name = class_mapping.get(class_number, 'unknown')

        # Create a directory for the class if it doesn't exist
        if(num%10 == 0):
          class_directory = os.path.join(validation_dir, class_name)
          os.makedirs(class_directory, exist_ok=True)
        else:
          class_directory = os.path.join(output_directory, class_name)
          os.makedirs(class_directory, exist_ok=True)

        # Copy the corresponding PNG file to the class directory
        png_file = os.path.splitext(txt_file)[0] + '.png'
        png_path = os.path.join(input_directory, png_file)
        shutil.copy(png_path, class_directory)
        num += 1

In [22]:
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

images_size = 224

In [23]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(images_size, images_size, 3))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [24]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    brightness_range=[0.4, 1.0],
    horizontal_flip = True,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
    rescale = 1./255)

train_gen = train_datagen.flow_from_directory(
    output_directory,
    # All images will be resized to 150x150
    target_size = (images_size, images_size),
    batch_size = 96,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode = 'sparse')

valid_gen = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (images_size, images_size),
    batch_size = 40,
    class_mode = 'sparse')

test_gen = test_datagen.flow_from_directory(
    "test",
    target_size = (images_size, images_size),
    batch_size = 20,
    class_mode = 'sparse')


Found 5760 images belonging to 8 classes.
Found 640 images belonging to 8 classes.
Found 1590 images belonging to 8 classes.


In [25]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

model.compile(loss= keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.RMSprop(learning_rate=0.0001),
              metrics=['acc'])

model.fit(train_gen,
            epochs=30,
            validation_data = valid_gen)

Epoch 1/30
60/60 [==============================] - 102s 2s/step - loss: 1.8533 - acc: 0.2790 - val_loss: 1.3180 - val_acc: 0.5156
Epoch 2/30
60/60 [==============================] - 102s 2s/step - loss: 1.2870 - acc: 0.5300 - val_loss: 0.6080 - val_acc: 0.8094
Epoch 3/30
60/60 [==============================] - 103s 2s/step - loss: 0.9199 - acc: 0.6760 - val_loss: 0.5876 - val_acc: 0.7719
Epoch 4/30
60/60 [==============================] - 101s 2s/step - loss: 0.6813 - acc: 0.7566 - val_loss: 0.3520 - val_acc: 0.8781
Epoch 5/30
60/60 [==============================] - 104s 2s/step - loss: 0.5669 - acc: 0.8066 - val_loss: 0.1452 - val_acc: 0.9438
Epoch 6/30
60/60 [==============================] - 100s 2s/step - loss: 0.4482 - acc: 0.8425 - val_loss: 0.4082 - val_acc: 0.8828
Epoch 7/30
60/60 [==============================] - 103s 2s/step - loss: 0.3962 - acc: 0.8601 - val_loss: 0.1142 - val_acc: 0.9500
Epoch 8/30
60/60 [==============================] - 100s 2s/step - loss: 0.3422 - a

In [26]:

model.evaluate(test_gen)

80/80 [==============================] - 16s 196ms/step - loss: 0.9212 - acc: 0.8585


[0.9211513996124268, 0.8584905862808228]

In [27]:
model.save_weights("Model.keras")